In [1]:
%run init.ipynb

Using TensorFlow backend.


matchzoo version 2.1.0

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
loading embedding ...
embedding loaded as `glove_embedding`


In [2]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=40, remove_stop_words=False)
train_pack_processed = preprocessor.fit_transform(train_pack_raw, drop_invalid=True)
valid_pack_processed = preprocessor.transform(dev_pack_raw, drop_invalid=False)
test_pack_processed = preprocessor.transform(test_pack_raw, drop_invalid=False)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 3778.26it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:07<00:00, 2458.38it/s]
Building Vocabulary from a datapack.: 100%|██████████| 404415/404415 [00:00<00:00, 1464884.40it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 3918.30it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:08<00:00, 2351.80it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 35354.76it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 3689.43it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00

In [3]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7ff029921908>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7ff02adb1cc0>,
 'vocab_size': 16675,
 'embedding_input_dim': 16675,
 'input_shapes': [(10,), (40,)]}

In [4]:
model = mz.models.KNRM()
model.params.update(preprocessor.context)
model.params['task'] = ranking_task
model.params['embedding_output_dim'] = glove_embedding.output_dim
model.params['embedding_trainable'] = True
model.params['kernel_num'] = 21
model.params['sigma'] = 0.1
model.params['exact_sigma'] = 0.001
model.params['optimizer'] = 'adadelta'
model.build()
model.compile()
#model.backend.summary()

In [5]:
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])
model.load_embedding_matrix(embedding_matrix)

In [6]:
pred_x, pred_y = test_pack_processed.unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_x))

In [7]:
train_generator = mz.DataGenerator(
    train_pack_processed,
    mode='pair',
    num_dup=5,
    num_neg=1,
    batch_size=20
)
print('num batches:', len(train_generator))

num batches: 255


In [8]:
history = model.fit_generator(train_generator, epochs=30, callbacks=[evaluate], workers=30, use_multiprocessing=True)

Epoch 1/30
255/255 [==============================] - 27s 106ms/step - loss: 0.9509
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5001188455279544 - normalized_discounted_cumulative_gain@5(0.0): 0.5607823759355992 - mean_average_precision(0.0): 0.5245636197131606
Epoch 2/30
255/255 [==============================] - 39s 153ms/step - loss: 0.5727
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5441281808636719 - normalized_discounted_cumulative_gain@5(0.0): 0.6045318662764055 - mean_average_precision(0.0): 0.5634624018715588
Epoch 3/30
255/255 [==============================] - 38s 150ms/step - loss: 0.4431
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5333453416480257 - normalized_discounted_cumulative_gain@5(0.0): 0.6003613586016829 - mean_average_precision(0.0): 0.5585197213894748
Epoch 4/30
255/255 [==============================] - 38s 148ms/step - loss: 0.3336
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.514923206680605 - norm